In [ ]:
import time
import json
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# Créez une instance de navigateur Chrome
driver = webdriver.Chrome()

In [3]:
fichier_json_url_Categories = 'Categories.json'

with open(fichier_json_url_Categories, 'r') as fichier_Categories:
    contenuCategories = fichier_Categories.read()  # Le code bug si je ne transforme pas le json en str en premier
    urlCategories = json.loads(contenuCategories)

In [4]:
# Configuration --------------------------------------------------------------------------------------------------------
driver.get('https://www.amazon.fr')
time.sleep(20)  # Au cas où il y aurait un kapcha à faire

In [ ]:
#  Boucle des catégories --------------------------------------------------------------------
for key, value in urlCategories.items():

    print(key)
    indicesPagesPasPrises2 = []
    #  Boucle des pages ------------------------------------------------------------------------------------------------
    
    # Recuperation des infos des livres des 6 première pages
    for i in range(1, 6):
        
        # Initialisation tableaux à chaque nouvelle page ---------------------------------------------------------------
        nom = []
        description = []
        photo = []
        isbn = []
        editeur = []
        prix = []
        categorie = []

        try:
            # On va sur chacune des pages
            driver.get(value.format(i, i))

            #  Page simple ---------------------------------------------------------------------------------------------
            # Je divise la recupération des liens en deux requêtes, car sinon les xpath est trop grand et l'IDE n'est pas content
            divPrincipal = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]')
            divs = divPrincipal.find_elements(By.CLASS_NAME, 's-widget-spacing-small')
            linksInPage = []  # tableau des liens des livres de la page actuelle

            for div in divs:  # recuperation des liens des livres de la page actuelle
                try:
                    elm = div.find_element(By.XPATH, './div/div/span/div/div/div/div[2]/div/div/div[3]/div[1]/div/div[1]/div[1]/a').text  # chemin relatif
                except:
                    elm = ""
                if elm == "":
                    try:
                        elm = div.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[3]/div/div/span/div/div/div/div[2]/div/div/div[3]/div[1]/div/div[1]/div[1]/a').text  # chemin complet
                    except:
                        elm = ""
                if elm == "":
                    try:
                        elm = div.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[9]/div/div/span/div/div/div/div[2]/div/div/div[3]/div[1]/div/div[1]/div[1]/a').text  # chemin complet
                    except:
                        elm = ""

                if elm == "Poche" or elm == "Relié" or elm == "Broché" or elm == "Carte":
                    try:
                        linksInPage.append(div.find_element(By.XPATH, './div/div/span/div/div/div/div[1]/div/div[2]/div/span/a').get_attribute('href'))  # chemin relatif
                    except:
                        linksInPage.append(div.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div/span[1]/div[1]/div[2]/div/div/div/div/span/div/div/div/div[1]/div/div[2]/div/span/a').get_attribute('href'))  # chemin complet
            print(i, ":", len(linksInPage), "--------------------------------------------------------------------------------------")

            cpt = 0  # tkt compteur pour l'affichage
            
            # Boucle des livres ----------------------------------------------------------------------------------------
            for link in linksInPage:
                
                cpt += 1  # tkt
                
                # On va sur chacune des pages des livres pour récupérer les infos qui nous interesse.
                driver.get(link)

                # Nom --------------------------------------------------------------------------------------------------
                try:
                    name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[9]/div[2]/div/h1/span[1]').text
                except:
                    name = ""
                if name == "":
                    try:
                        name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[7]/div[2]/div/h1/span[1]').text
                    except:
                        name = ""
                if name == "":
                    try:
                        name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[9]/div[2]/div/h1/span[1]').text
                    except:
                        name = ""
                if name == "":
                    try:
                        name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[8]/div[2]/div/h1/span[1]').text
                    except:
                        name = ""
                name = name.replace("'", "&#39")
                nom.append(name)

                # Description ------------------------------------------------------------------------------------------
                try:
                    desc = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[8]/div[28]/div[1]/div/div[1]').text
                except:
                    desc = ""
                if desc == "":
                    try:
                        desc = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[7]/div[28]/div/div[1]').text
                    except:
                        desc = ""
                if desc == "":
                    try:
                        desc = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[8]/div[27]/div/div[1]').text
                    except:
                        desc = ""
                if desc == "":
                    try:
                        desc = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[7]/div[27]/div/div[1]').text
                    except:
                        desc = ""
                if desc == "":
                    try:
                        desc = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div[9]/div[27]/div/div[1]/span[1]').text
                    except:
                        desc = ""
                desc = desc.replace("'", "&#39")
                description.append(desc)

                #  Photo -----------------------------------------------------------------------------------------------
                try:
                    pic = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[7]/div[1]/div[1]/div/div/div/div[1]/div[1]/ul/li[1]/span/span/div/img').get_attribute('src')
                except:
                    pic = ""
                if pic == "":
                    try:
                        pic = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[4]/div[1]/div[6]/div[1]/div[1]/div/div/div/div[1]/div[1]/ul/li[1]/span/span/div/img').get_attribute('src')
                    except:
                        pic = ""
                if pic == "":
                    try:
                        pic = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div[8]/div[1]/div[1]/div/div/div/div[1]/div[1]/ul/li[1]/span/span/div/img').get_attribute('src')
                    except:
                        pic = ""
                photo.append(pic)

                # Detail -----------------------------------------------------------------------------------------------
                Isbn = ""
                edit = ""
                try:
                    details = driver.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[21]/div/div[1]/ul/li')
                    for det in details:
                        truc = det.find_element(By.XPATH, './span/span[1]').text
                        if truc.__contains__("ISBN-13"):
                            Isbn = det.find_element(By.XPATH, './span/span[2]').text
                        elif truc.__contains__("Éditeur"):
                            edit = det.find_element(By.XPATH, './span/span[2]').text
                except:
                    pass
                Isbn = Isbn.replace("'", "&#39")
                edit = edit.replace("'", "&#39")
                isbn.append(Isbn)
                editeur.append(edit)

                #  Prix ------------------------------------------------------------------------------------------------
                price = 0.0
                try:
                    priceInt = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[2]/div/form/div/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[1]').text
                    priceFloat = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[2]/div/form/div/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[2]').text
                    price = priceInt + "." + priceFloat
                    price = float(price)
                except:
                    price = 0.0
                if price == 0.0:
                    try:
                        priceInt = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div/div/form/div/div/div/div/div[4]/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[1]').text
                        priceFloat = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div/div/form/div/div/div/div/div[4]/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[2]').text
                        price = priceInt + "." + priceFloat
                        price = float(price)
                    except:
                        price = 0.0
                if price == 0.0:
                    try:
                        priceInt = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div[2]/div/div/div/div/div/form/div/div/div/div/div[4]/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[1]').text
                        priceFloat = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[4]/div/div[1]/div/div[2]/div/div/div/div/div/form/div/div/div/div/div[4]/div/div[2]/div[1]/div[1]/span[2]/span[2]/span[2]').text
                        price = priceInt + "." + priceFloat
                        price = float(price)
                    except:
                        price = 0.0
                if price == 0.0:
                    try:
                        priceSTR = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[2]/div/div/div/div[2]/span/span/a/span[2]/span')
                        priceSTR = priceSTR[:-2]
                        priceSTR = priceSTR.replace(',', '.')
                        price = float(priceSTR)
                    except:
                        price = 0.0
                if price == 0.0:
                    try:
                        priceSTR = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[4]/div[1]/div[5]/div[4]/div[2]/div/div/div[2]/div[2]/span/span/a/span[2]/span')
                        priceSTR = priceSTR[:-2]
                        priceSTR = priceSTR.replace(',', '.')
                        price = float(priceSTR)
                    except:
                        price = 0.0
                if price == 0.0:
                    nombre_aleatoire = np.random.uniform(5.0, 20.0)
                    price = round(nombre_aleatoire, 2)
                prix.append(price)

                # Catégorie --------------------------------------------------------------------------------------------
                categorie.append(key)

                print(cpt, "/", len(linksInPage))

            #  Ajout des données dans la CSV ---------------------------------------------------------------------------
            if len(nom) == len(description) == len(photo) == len(isbn) == len(editeur) == len(prix) == len(categorie):
                time.sleep(1)
                dfLivres = pd.DataFrame(
                    {"Nom": nom, "Prix": prix, "Description": description, "Isbn": isbn, "Photo": photo,
                     "Editeur": editeur, "Categorie": categorie})
                fileNameLivres = 'CSV2/Livres' + key + '.csv'
                if i == 1:
                    dfLivres.to_csv(fileNameLivres, index=False, encoding='utf-8')
                else:
                    dfLivres.to_csv(fileNameLivres, mode='a', index=False, header=False, encoding='utf-8')
            else:
                print("Len livres pas OK")
                print("Nom", len(nom))
                print("Description", len(description))
                print("Photo", len(photo))
                print("ISBN", len(isbn))
                print("Editeur", len(editeur))
                print("Prix", len(prix))
                print("Categorie", len(categorie))
        except:
            print("PB recupération des liens des livres")

# Temps : 31m 16.2s

Biographies
1 : 14 --------------------------------------------------------------------------------------
1 / 14
2 / 14
3 / 14
4 / 14
5 / 14
6 / 14
7 / 14
8 / 14
9 / 14
10 / 14
11 / 14
12 / 14
13 / 14
14 / 14
2 : 11 --------------------------------------------------------------------------------------
1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11
3 : 11 --------------------------------------------------------------------------------------
1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11
4 : 15 --------------------------------------------------------------------------------------
1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15
5 : 15 --------------------------------------------------------------------------------------
1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15
Poesie
1 : 14 ---------------------

In [6]:
# Fermer le navigateur
driver.quit()
print("Le scrap est fini !!!!!!!!!!!!!!!!!")

Le scrap est fini !!!!!!!!!!!!!!!!!
